# <center> DETEKSI POLA AMBIGUITAS STRUKTURAL </center>

## 1. Import Library NLTK RegexpParser & Pandas

In [10]:
import nltk
from nltk import RegexpParser
import pandas as pd

## 2. Import Dokumen Anotasi Pernyataan Kebutuhan Perangkat Lunak

In [11]:
def open_file(nameFile):
    with open(nameFile, encoding="utf-8") as reqs:
        data_reqs = reqs.readlines()
    all_reqs = [x.rstrip('\n') for x in data_reqs]
    return all_reqs

In [ ]:
inputreadfile = input()
readFile = "Hasil Tag/Tag "+inputreadfile+".txt"
requirements = open_file(readFile)

## 3. Sentence Splitting

In [ ]:
sentences= []
tags = []
kalimat = []
index = 0
while index < len(requirements):
    sent = [] #untuk menampung katanya
    kata = []
    tag = [] #untuk menampung postag dari tiap kata
    while not requirements[index].startswith('</kalimat'): #batas akhir dari kalimat
        if  not requirements[index].startswith('<kalimat'):
            content_part = requirements[index].split('\t')
            sent.append((content_part[0],content_part[1]))
            kata.append(content_part[0])
            tag.append(content_part[1])
        index = index + 1
    sentences.append(sent)
    kalimat.append(kata)
    tags.append(tag)
    index = index+2

## 4. Pernyataan Kebutuhan Perangkat Lunak dan Tag-nya

In [ ]:
i = 0
for sentence in sentences:
    print(i,sentence)
    i+=1

## 5. Analisis Vague Words

In [ ]:
dictionaryAmbiguity = {}
vaguefile = open_file('vague words.txt')
print(vaguefile)
countVague=0
for i in range(len(kalimat)):
    for kata in kalimat[i]:
        if kata in vaguefile:
            print(i,kalimat[i],kata)
            if i in dictionaryAmbiguity:
                dictionaryAmbiguity[i] += ', vague'
            else:
                dictionaryAmbiguity[i] = 'vague'
            countVague+=1
print(dictionaryAmbiguity)

In [ ]:
# countMUoM = 0
# for i in range(len(tags)-1):
#     for j in range(len(tags[i])-1):
#         if tags[i][j] == 'CD':
#             if tags[i][j-1]=='NN': continue
#             elif tags[i][j+1]=='NN' or tags[i][j+1]=='SYM':continue
#             elif tags[i][j+1] != 'NN' or tags[i][j+1]!='SYM': 
#                 print(i,sentences[i])
#                 if i in dictionaryAmbiguity:
#                     dictionaryAmbiguity[i] += ', missing unit measurement'
#                 else:
#                     dictionaryAmbiguity[i] = 'missing unit measurement'
#                 countMUoM+=1
#         else: continue
# print(dictionaryAmbiguity)

## 6. Shallow Parsing

In [ ]:
polaAmbiguity = []
pola = """
analytical:{<NN.*><NN.*><JJ.*>}
coordination:{<NN.*><CC><NN.*><JJ>}
PPattachment:{<VB><NN.*>*<JJ><IN><NN.*><JJ>}
"""
chunk = RegexpParser(pola)
for kalimats in sentences:
    result = chunk.parse(kalimats)
    polaAmbiguity.append(str(result).split("\n"))

In [ ]:
kataAnalytical = "analytical"
kataPPattachment = "PPattachment"
kataCoordination = "coordination"
countAnalytical = 0
countCoordination = 0
countPPattachment = 0
for i in range(len(polaAmbiguity)):
    for j in polaAmbiguity[i]:
        if (kataAnalytical in j):
            print(i,polaAmbiguity[i])
            if i in dictionaryAmbiguity:
                dictionaryAmbiguity[i] += ', analytical'
            else:
                dictionaryAmbiguity[i] = 'analytical'
            countAnalytical+=1
        if (kataCoordination in j):
            print(i,polaAmbiguity[i])
            if i in dictionaryAmbiguity:
                dictionaryAmbiguity[i] += ', coordination'
            else:
                dictionaryAmbiguity[i] = 'coordination'
            countCoordination+=1
        if (kataPPattachment in j):
            print(i,polaAmbiguity[i])
            if i in dictionaryAmbiguity:
                dictionaryAmbiguity[i] += ', PPattachment'
            else:
                dictionaryAmbiguity[i] = 'PPattachment'
            countPPattachment+=1
print(dictionaryAmbiguity)

## 7. Data Hasil Analisis Jenis Ambiguitas

In [ ]:
listSentences = []
for i in range(len(sentences)):
    if i in dictionaryAmbiguity.keys():
        print(i)
        listSentences.append((sentences[i],dictionaryAmbiguity[i]))
    else:
        listSentences.append((sentences[i],'-'))

In [ ]:
print("Jumlah Kata Vague: ",countVague)
# print("Jumlah Missing Unit of Measurement: ",countMUoM)
print("Jumlah Ambiguitas Analitis: ",countAnalytical)
print("Jumlah Ambiguitas Corrdination: ",countCoordination)
print("Jumlah Ambiguitas PP Attachment: ",countPPattachment)
totalAmbiguitasStruktural = countVague+countAnalytical+countCoordination+countPPattachment
print("Total Ambiguitas Struktural dalam dokumen: ",totalAmbiguitasStruktural)

Jumlah Kata Vague:  0
Jumlah Ambiguitas Analitis:  2
Jumlah Ambiguitas Corrdination:  2
Jumlah Ambiguitas PP Attachment:  0
Total Ambiguitas Struktural dalam dokumen:  4


## 8. Transformasi Data ke Dokumen

In [ ]:
raw_data = {
            'Pernyataan Kebutuhan PL': [kalimat[i] for i in range(len(kalimat))],
            'Urutan Tag Kebutuhan PL': [tags[i] for i in range(len(tags))],
            'Jenis Ambiguitas': [listSentences[i][1] for i in range(len(listSentences))]
            }
df = pd.DataFrame(raw_data,columns=['Pernyataan Kebutuhan PL','Urutan Tag Kebutuhan PL','Jenis Ambiguitas'])
fileHasilAnalisis="Hasil Analisis/Analisis "+inputreadfile+".csv"
df.to_csv(fileHasilAnalisis,encoding='utf-8')
df

,Pernyataan Kebutuhan PL,Jenis Ambiguitas
0,"[Sistem, dapat, menampilkan, halaman, login, y...",coordination
1,"[Sistem, menampilkan, page, login, ,, dengan, ...",-
2,"[Sistem, menampilkan, page, login, yang, dilen...",-
3,"[Sistem, menampilkan, page, login, hanya, untu...",-
4,"[Sistem, dapat, mencatat, waktu, dan, ip, dari...",-
5,"[Sistem, dapat, mencatat, dan, menyimpan, ke, ...",-
6,"[Sistem, dapat, mencatat, dan, menyimpan, wakt...",-
7,"[Sistem, dapat, mencatat, dan, menyimpan, wakt...",-
8,"[Sistem, dapat, mencatat, dan, menyimpan, tota...",-
9,"[Sistem, dapat, menampilkan, pesan, kesalahan,...",analytical
